## Fourtify using Fopy for OD
Use Find_Orb to solve an orbit for a 3-nighter via ***Fopy*** and then search diaSources for more detections along that orbit with ***Fourtify***.  You need to install ***Fopy*** from [https://github.com/bengebre/fopy](https://github.com/bengebre/fopy) first to run this code.

**Credit**: The DP0.3 data set was generated by members of the Rubin Solar System Pipelines and Commissioning teams, with help from the LSST Solar System Science Collaboration, in particular: Pedro Bernardinelli, Jake Kurlander, Joachim Moeyens, Samuel Cornwall, Ari Heinze, Steph Merritt, Lynne Jones, Siegfried Eggl, Meg Schwamb, Grigori Fedorets, and Mario Juric.

## Imports & Helpers

In [ ]:
import numpy as np
import pandas as pd
import json
from fourtify import Fourtify
from fopy import Fopy
from astropy.time import Time
from astropy import units as u
from lsst.rsp import get_tap_service

In [2]:
def query(adql, con=None):
    """
    DB query
    """
    if con is None:
        con = get_tap_service("ssotap")
    return con.search(adql).to_table().to_pandas()

In [3]:
def create_observations(time_ranges,dist_ranges):
    """
    Query DP0.3 data for observations in a time range and distance range.
    """

    q = """
        SELECT ds.ra,ds.dec,ds.midPointMjdTai,ds.diaSourceId,ds.ssObjectId,
        sss.heliocentricDist,
        sss.heliocentricX-sss.topocentricX as observerX,
        sss.heliocentricY-sss.topocentricY as observerY,
        sss.heliocentricZ-sss.topocentricZ as observerZ
        FROM dp03_catalogs_1yr.DiaSource AS ds
        INNER JOIN dp03_catalogs_1yr.SSSource AS sss
        ON ds.diaSourceId = sss.diaSourceId
        WHERE ds.midPointMjdTai >= {:f} AND ds.midPointMjdTai <= {:f} 
        AND sss.heliocentricDist > {:f} AND sss.heliocentricDist <= {:f}
        ORDER BY midPointMjdTai,ds.diaSourceId ASC
    """
    
    obs = query(q.format(time_ranges[0],time_ranges[1],dist_ranges[0],dist_ranges[1]))
    obs[['observerX','observerY','observerZ']] = obs[['observerX','observerY','observerZ']].astype(float)
                
    return obs

In [4]:
def parse_json(data):
    obj = data['objects'][list(data['objects'].keys())[0]]
    fo_rms = obj['elements']['rms_residual']
    fo_used = obj['observations']['used']
    fo_count = obj['observations']['count']
    a = obj['elements']['a']
    e = obj['elements']['e']
    i = obj['elements']['i']
    node = obj['elements']['asc_node']
    peri = obj['elements']['arg_per']
    M = obj['elements']['M']
    epoch = obj['elements']['epoch']

    solved = fo_used==fo_count

    return solved,fo_rms,a,e,i,node,peri,M,epoch

## Load data from diaSources

In [5]:
#get all DP0.3 transient sources in a 45 day window from database 
#NOTE: this takes like 10-15 minutes because we're loading ~5.9MM sources!

time_interval = [60218.00491,60218.00491+45] #mjd interval to get diaSources on
range_interval = [0,1e10] #all distances (AU)
obs = create_observations(time_interval,range_interval)

In [6]:
#these are a subset of observations for an object that we'll calculate an orbit for and try to extend
obj = obs.loc[[3275344, 3323484, 3616951, 3684100, 3972996, 4017229]]

## Calling Fopy & Fourtify

In [7]:
#initialize Fopy solve directory (only have to initialize once)
#reset=True resets index counter to zero and removes prior solves from OD/
fp = Fopy('./OD',reset=True)

#setup inputs for OD solve - times are UTC Julian dates
fo_radecs = obj[['ra','dec']].values
fo_times = Time(obj[['midPointMjdTai']],format='mjd',scale='utc').jd.flatten()
fo_obs_ids = ['I11'] * len(fo_times)

#solve orbit with Fopy then load and parse solution
mpc_file = fp.write(fo_radecs,fo_times,fo_obs_ids)
json_file = fp.solve(mpc_file)
data = fp.load_json(json_file)
solved,rms,a,e,i,node,peri,M,epoch = parse_json(data)
solved,rms

(True, 0.02985)

In [8]:
#setup diaSources data for Fourtify - times are TDB Julian dates
#this is slow, but you only have to generate and initialize the transient data once
db_radecs = obs[['ra','dec']].values
db_times = Time(obs['midPointMjdTai'],format='mjd',scale='utc',location=(289.26345*u.deg,-30.240641*u.deg)).tdb.jd.flatten()
db_locs = obs[['observerX','observerY','observerZ']].values

#initialize Fourtify with diaSources data
ff = Fourtify(db_radecs,db_times,db_locs)

In [9]:
#look for more transient sources on this orbit with Fourtify
#(10,1,0): 10" max deviation at rate of 1"/day with starting deviation of 0"
dradecs,fidx = ff.orbit((a,e,i,node,peri,M),epoch,(10,1,0))

In [10]:
#the additional transient sources found by Fourtify (note they all have the same ssObjectId)
obs.loc[fidx]

,ra,dec,midPointMjdTai,diaSourceId,ssObjectId,heliocentricDist,observerX,observerY,observerZ
2263441,20.623939,-2.110646,60229.20047,4663528428262270326,-7978867435213711595,1.862491,0.947886,0.287306,0.124508
2325433,20.618018,-2.112435,60229.22449,-5880678224015631640,-7978867435213711595,1.862475,0.947747,0.287670,0.124663
2572086,20.392969,-2.183249,60230.19580,1652531757451060663,-7978867435213711595,1.861845,0.942096,0.302128,0.130934
2634741,20.387017,-2.184967,60230.21986,-8074616670269294953,-7978867435213711595,1.861830,0.941950,0.302490,0.131088
3275344,19.712111,-2.379780,60233.11936,5779554338486750538,-7978867435213711595,1.860026,0.923483,0.345116,0.149577
3323484,19.706260,-2.381273,60233.14308,-666928304096274332,-7978867435213711595,1.860012,0.923322,0.345467,0.149727
3616951,19.017952,-2.551895,60236.11443,-7264436804714889325,-7978867435213711595,1.858282,0.901960,0.388251,0.168277
3684100,19.012112,-2.553141,60236.13829,7358906051747058725,-7978867435213711595,1.858269,0.901778,0.388596,0.168424
3972996,18.551063,-2.650573,60238.16414,5585262173597708038,-7978867435213711595,1.857160,0.885827,0.417182,0.180814
4017229,18.545327,-2.651638,60238.18795,-4090268930407646038,-7978867435213711595,1.857147,0.885631,0.417520,0.180958
